We want to isolate households with two parents living under civil union with children being over (or not) 21/25 yo.

In [99]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2019-01-31 14:57:04.234612


In [100]:
%matplotlib inline

In [101]:
secret = False #Will be put to True for a CASD export

In [102]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle

In [103]:
import pickle
pickle_year_variable_path = "C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/Program_only_married_couples/year_variable/"
year =  pickle.load(open(pickle_year_variable_path+"year_variable.p",'rb')); print(year)

2011


In [104]:
store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5'
reaction_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/reaction.h5'
pickles_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/pickles/{}'.format(year)
Store = pd.HDFStore(store_path)

In [105]:
Store

<class 'pandas.io.pytables.HDFStore'>
File path: C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5

In [106]:
for key in Store.keys(): print(key)
Store.close()

/DECES
/DESCENDANCE
/EAR2004_FAMILLE
/EAR2004_INDIVIDU
/EAR2004_LOGEMENT
/EAR2005_FAMILLE
/EAR2005_INDIVIDU
/EAR2005_LOGEMENT
/EAR2006_FAMILLE
/EAR2006_INDIVIDU
/EAR2006_LOGEMENT
/EAR2007_FAMILLE
/EAR2007_INDIVIDU
/EAR2007_LOGEMENT
/EAR2008_FAMILLE
/EAR2008_INDIVIDU
/EAR2008_LOGEMENT
/EAR2009_FAMILLE
/EAR2009_INDIVIDU
/EAR2009_LOGEMENT
/EAR2010_FAMILLE
/EAR2010_INDIVIDU
/EAR2010_LOGEMENT
/EAR2011_FAMILLE
/EAR2011_INDIVIDU
/EAR2011_LOGEMENT
/EAR2012_FAMILLE
/EAR2012_INDIVIDU
/EAR2012_LOGEMENT
/EAR2013_FAMILLE
/EAR2013_INDIVIDU
/EAR2013_LOGEMENT
/EAR2014_FAMILLE
/EAR2014_INDIVIDU
/EAR2014_LOGEMENT
/EAR2015_FAMILLE
/EAR2015_INDIVIDU
/EAR2015_LOGEMENT
/EAR2016_FAMILLE
/EAR2016_INDIVIDU
/EAR2016_LOGEMENT
/FE
/FISC_INDIVIDU_2011
/FISC_INDIVIDU_2012
/FISC_INDIVIDU_2013
/FISC_INDIVIDU_2014
/FISC_INDIVIDU_2015
/FISC_INDIVIDU_2016
/FISC_LOGEMENT_2011
/FISC_LOGEMENT_2012
/FISC_LOGEMENT_2013
/FISC_LOGEMENT_2014
/FISC_LOGEMENT_2015
/FISC_LOGEMENT_2016
/FISC_REVDET_2011
/FISC_REVDET_2012
/FISC_REVDE

In [107]:
fisc_revenu_2011 = pd.read_hdf( store_path,"FISC_REVENU_{}".format(2011+1) )
fisc_revenu_2014 = pd.read_hdf( store_path,"FISC_REVENU_{}".format(2014+1) )

In [108]:
fiscr = pd.merge(fisc_revenu_2011, fisc_revenu_2014, on = "ID_FISC_LOG_DIFF", 
                 suffixes = ("_2011", "_2014"))

In [109]:
fiscr.shape

(1741300, 199)

## Data selection

We keep households with only one fiscal-household inside the household.  
We keep households with TYPMEN9 = 30 41,42,43 in both years (respectively married couples with 0,1,2, and 
3 or more children)in 2011 and 2014.

In [110]:
# We keep households with only one fiscal-household inside the household.

fiscr = fiscr[fiscr.NBFOYM_2011 == fiscr.NBFOYM_2014]
fiscr = fiscr[fiscr.NBFOYM_2011 == 1]

In [111]:
fiscr.shape

(1137027, 199)

In [112]:
#We keep households with TYPMEN9 = 30 41,42,43 in both years

fiscr = fiscr[fiscr.TYPMEN9_2011.isin([30,41,42,43])]
fiscr = fiscr[fiscr.TYPMEN9_2014.isin([30,41,42,43])]

In [113]:
fiscr.shape

(811940, 199)

In [114]:
## Check that we only have Two non dependent persons
(fiscr.NBPERSM_2011- fiscr.NBPAC_2011).value_counts()

2.0    811940
dtype: int64

We drop households with children under shared residence.

In [115]:
fiscr = fiscr[(fiscr.NBERAM_2011 == 0)&(fiscr.NBERAM_2014 == 0)]

In [116]:
fiscr.shape

(808324, 199)

Load the fisc_individu table that contains individual informations

In [117]:
## Fisci
fisc_individu_2011 = pd.read_hdf( store_path,"FISC_INDIVIDU_{}".format(2011+1))

In [118]:
## Fisci
fisc_individu_2014 = pd.read_hdf( store_path,"FISC_INDIVIDU_{}".format(2014+1))

We only keep individuals that belong to households present in the `fiscr` `DataFrame` (fiscal households with tax-fillers under civil union).

In [119]:
id_fisc_log_fiscr = fiscr.ID_FISC_LOG_DIFF

In [120]:
fisci_2011 = fisc_individu_2011[fisc_individu_2011.ID_FISC_LOG_DIFF.isin(id_fisc_log_fiscr)]

In [121]:
fisci_2014 = fisc_individu_2014[fisc_individu_2014.ID_FISC_LOG_DIFF.isin(id_fisc_log_fiscr)]

In [122]:
print(fisci_2011.shape, fisci_2014.shape)

(2599801, 32) (2621068, 32)


We  have 2.6 millions individuals, in 2011 and 2014, including parents and children, living in households with couples under civil union (Mariage/Pacs).

The individual variable `TYPE_FISC` gives the fiscal type of an individual.
* 1 correspond to the main tax filler
* 2 correspond to the second tax filler (second spouse)
* A,B,C,D, etc. correspond to minor children dependent of the tax household
* 3,4,5,6, etc. correspond to major children still dependent of the tax household 


For instance a fiscal household for which the aggregated values of `TYPE_FISC` is 123AB correspond to a household with two parents/tax fillers, one dependent child over 18 (being a dependent child over 18 is discretionary to the child and must have a student status after 21 yo, after 25 yo children can not be dependent (on their parent tax return anymore). There exists exception for children with disabilities.

In [123]:
#TODO: nettoyer
## Garder uniquement 1234ABCD
fisci_2011[["TYPE_FISC", "ID_FISC_LOG_DIFF"]].head(10000).groupby('ID_FISC_LOG_DIFF').sum().TYPE_FISC.value_counts()[4:15]

12ABCD     79
123        62
123A       49
123AB      34
1234       18
12B        18
12ABCDE    12
1234A       8
12BCD       7
1212        6
123ABC      6
Name: TYPE_FISC, dtype: int64

We see that there exists household that do not match our definition. One reson is the they are not present in the `FISC_Individu` database (for instance when dependent children do not live in the household anymore?).

We are going to clean the household that we keep based on the discrepencies between the `FISC_INDIVIDU` database and the `FISC_REVENU` database.

In [124]:
%%time
typ_fisc_grpby_2011 = fisci_2011[["TYPE_FISC", "ID_FISC_LOG_DIFF"]]\
            .groupby('ID_FISC_LOG_DIFF').TYPE_FISC.apply("".join)

Wall time: 35.2 s


In [125]:
%%time
typ_fisc_grpby_2014 = fisci_2014[["TYPE_FISC", "ID_FISC_LOG_DIFF"]]\
            .groupby('ID_FISC_LOG_DIFF').TYPE_FISC.apply("".join)

Wall time: 35.3 s


In [126]:
typ_fisc_grpby = pd.merge(
        pd.DataFrame(typ_fisc_grpby_2011).reset_index(), 
        pd.DataFrame(typ_fisc_grpby_2014).reset_index(),
        suffixes = ("_2011", "_2014"), how = "inner", on = "ID_FISC_LOG_DIFF")

In [127]:
typ_fisc_grpby.head()

,ID_FISC_LOG_DIFF,TYPE_FISC_2011,TYPE_FISC_2014
0,4.0,123A,1234
1,6.0,12,12
2,11.0,12,12
3,12.0,123,12
4,20.0,12ABC,12ABC


In [128]:
typ_fisc_grpby.shape

(808324, 3)

In [129]:
%%time
fiscr = pd.merge(fiscr, pd.DataFrame(typ_fisc_grpby).reset_index(), on = "ID_FISC_LOG_DIFF", how = "inner")

Wall time: 14.1 s


In [130]:
### We select only those with a unique tax-filler (déclarant) and a unique spouse (conjoint)
id_drop_more_than_one_declarant_2011 = typ_fisc_grpby[typ_fisc_grpby.TYPE_FISC_2011.str.count('1') != 1].index
id_drop_more_than_one_conjoint_2011 = typ_fisc_grpby[typ_fisc_grpby.TYPE_FISC_2011.str.count('2') != 1].index



In [131]:
### We select only those with a unique tax-filler (déclarant) and a unique spouse (conjoint)
id_drop_more_than_one_declarant_2014 = typ_fisc_grpby[typ_fisc_grpby.TYPE_FISC_2014.str.count('1') != 1].index
id_drop_more_than_one_conjoint_2014 = typ_fisc_grpby[typ_fisc_grpby.TYPE_FISC_2014.str.count('2') != 1].index



In [132]:
print(id_drop_more_than_one_declarant_2011.shape,id_drop_more_than_one_declarant_2011.shape) 
print(id_drop_more_than_one_declarant_2014.shape,id_drop_more_than_one_conjoint_2014.shape) 

(5686,) (5686,)
(4587,) (4482,)


In [133]:
id_drop_more_than_one_declarant = set(
    ).union(*[id_drop_more_than_one_declarant_2011,id_drop_more_than_one_declarant_2011,
             id_drop_more_than_one_declarant_2014,id_drop_more_than_one_conjoint_2014
             ]
           )

In [134]:
len(id_drop_more_than_one_declarant)

8158

In [135]:
fiscr = fiscr[~ fiscr.ID_FISC_LOG_DIFF.isin(id_drop_more_than_one_declarant )]
fiscr.shape

(806660, 202)

### Check that the number of individual in fisci is the same than in fiscr

In [136]:
(fiscr.TYPE_FISC_2011.str.len()== fiscr.NBPERSM_2011).value_counts()

True     796542
False     10118
dtype: int64

In [137]:
(fiscr.TYPE_FISC_2014.str.len()== fiscr.NBPERSM_2014).value_counts()

True     798033
False      8627
dtype: int64

#### Drop households for which it is not the case

In [138]:
fiscr = fiscr[
     (fiscr.TYPE_FISC_2011.str.len()== fiscr.NBPERSM_2011)
    &(fiscr.TYPE_FISC_2014.str.len()== fiscr.NBPERSM_2014)
    ] 

In [139]:
fiscr.shape

(790495, 202)

In [140]:
fiscr.TYPE_FISC_2011.value_counts()[:10]

12        342257
12AB      193003
12A        93864
12ABC      85878
12ABCD     17798
123        14286
123A       12209
123AB       7486
12B         4132
1234        3977
Name: TYPE_FISC_2011, dtype: int64

Why do we have sometimes 12B while 12A is expected (deaths of child A ?). # TODO

In [141]:
### C'est la table que l'on va garder
fiscr.describe()

,AN_FISC_2011,AN_REVENU_2011,REG_2011,TU10_2011,NBFOYM_2011,NBPERSM_2011,INF14M_2011,SUP14M_2011,NBERAM_2011,NB_UC_2011,AGERF_2011,SEXERF_2011,OCCTYP_2011,TYPMEN9_2011,NB_ALLOC_CNAF_2011,NB_ALLOC_CNAV_2011,NB_ALLOC_MSAF_2011,NB_ALLOC_MSAV_2011,NIVVIEM_2011,CENTILE_2011,REVDISPM_2011,REVPERM_2011,REVINIM_2011,REVDECM_2011,ZTSAM_2011,ZSALM_2011,ZCHOM_2011,ZPERM_2011,ZRETM_2011,ZRSTM_2011,ZALRM_2011,ZRTOM_2011,ZRAGM_2011,ZRICM_2011,ZRNCM_2011,ZFONM_2011,ZVAMM0_2011,ZVAMM_2011,ZVALM0_2011,ZVALM_2011,ZRACM_2011,ZETRM_2011,ZALVM_2011,PSOCM_2011,PFAMM_2011,M_PNAM_2011,M_PAJEM_2011,M_CLCAM_2011,M_COLCAM_2011,M_AFM_2011,M_CFM_2011,M_ARSM_2011,M_AEEHM_2011,M_ASFM_2011,MINIM_2011,M_MINVM_2011,M_RSAM_2011,M_RSA_ACTM_2011,M_RSA_SOCM_2011,M_RSA_PFAM_2011,M_AAH_CAAHM_2011,LOGTM_2011,M_ALFM_2011,M_APLM_2011,M_ALSM_2011,M_ALSETM_2011,M_PPEM_2011,ZIMPOM_2011,ZIMPVALM_2011,ZTHABM_2011,CSGIM_2011,CSGISALM_2011,CRDSM_2011,CRDSSALM_2011,CSGPATM_2011,CSGVALM_2011,CSGIMPM_2011,CSGDM_2011,CSGDSALM_2011,PRODUITFIN_2011,ZPPEM_2011,ZQUOM_2011,ZDIVM_2011,ZGLOM_2011,ZAVFM_2011,ZIMPFORM_2011,NBPAC_2011,GARDEM_2011,SERVDOMM_2011,ZIMPOM_DECL_2011,ZIMPOM_ANC_2011,ZIMPOM_ANC_DECL_2011,ZAVFM_DECL_2011,ZTHABM_DECL_2011,AN_FISC_2014,AN_REVENU_2014,REG_2014,TU10_2014,NBFOYM_2014,NBPERSM_2014,INF14M_2014,SUP14M_2014,NBERAM_2014,NB_UC_2014,AGERF_2014,SEXERF_2014,OCCTYP_2014,TYPMEN9_2014,I_CHAMPM_2014,NB_ALLOC_CNAF_2014,NB_ALLOC_CNAV_2014,NB_ALLOC_MSAF_2014,NB_ALLOC_MSAV_2014,NIVVIEM_2014,CENTILE_2014,REVDISPM_2014,REVPERM_2014,REVINIM_2014,REVDECM_2014,ZTSAM_2014,ZSALM_2014,ZCHOM_2014,ZPERM_2014,ZRETM_2014,ZRSTM_2014,ZALRM_2014,ZRTOM_2014,ZRAGM_2014,ZRICM_2014,ZRNCM_2014,ZFONM_2014,ZVAMM0_2014,ZVAMM_2014,ZVALM0_2014,ZVALM_2014,ZRACM_2014,ZETRM_2014,ZALVM_2014,PSOCM_2014,PFAMM_2014,M_PNAM_2014,M_PAJEM_2014,M_CLCAM_2014,M_COLCAM_2014,M_AFM_2014,M_CFM_2014,M_ARSM_2014,M_AEEHM_2014,M_ASFM_2014,MINIM_2014,M_MINVM_2014,M_RSAM_2014,M_RSA_ACTM_2014,M_RSA_SOCM_2014,M_RSA_PFAM_2014,M_AAH_CAAHM_2014,LOGTM_2014,M_ALFM_2014,M_APLM_2014,M_ALSM_2014,M_ALSETM_2014,M_PPEM_2014,ZIMPOM_2014,ZIMPVALM_2014,ZTHABM_2014,CSGIM_2014,CSGISALM_2014,CRDSM_2014,CRDSSALM_2014,CSGPATM_2014,CSGVALM_2014,CSGIMPM_2014,CSGDM_2014,CSGDSALM_2014,PRODUITFIN_2014,ZPPEM_2014,ZQUOM_2014,ZDIVM_2014,ZGLOM_2014,ZAVFM_2014,ZIMPFORM_2014,NBPAC_2014,GARDEM_2014,SERVDOMM_2014,ZIMPOM_DECL_2014,ZIMPOM_ANC_2014,ZIMPOM_ANC_DECL_2014,ZAVFM_DECL_2014,ZTHABM_DECL_2014,index
count,790495.0,790495.0,790495.000000,790495.000000,790495.0,790495.000000,790495.000000,790495.000000,790495.0,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,780371.000000,780371.000000,780371.000000,780371.000000,7.803710e+05,780371.000000,7.803710e+05,7.803710e+05,7.803710e+05,7.803710e+05,7.904950e+05,7.904950e+05,790495.000000,7.904950e+05,7.904950e+05,7.904950e+05,790495.000000,790495.000000,7.904950e+05,7.904950e+05,7.904950e+05,7.803710e+05,7.803710e+05,7.803710e+05,7.803710e+05,7.803710e+05,7.803710e+05,7.904950e+05,790495.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.00000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,780371.000000,7.904950e+05,7.803710e+05,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,7.904950e+05,790495.000000,790495.000000,790495.000000,780371.000000,780371.000000,7.904950e+05,7.803710e+05,7.803710e+05,780371.000000,7.904950e+05,790495.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,790495.0,790495.0,790495.000000,790495.000000,790495.0,790495.000000,790495.000000,790495.000000,790495.0,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,790495.000000,7.803710e+05,780371.000000,7.803710e+05,7.803710e+05,7.803710e+05,7.904950e+05,7.904950e+05,7.904950e+05,790495.000000,790495.0

In [142]:
result_path = u"C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/Program_only_married_couples/hdf_married_couples/"
fiscr.to_hdf(result_path+"married_couples_step_1.h5", "fiscr")

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\pandas\core\generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['ID_FISC_LOG_DIFF', 'DEP_2011', 'DEPCOM_2011', 'I_CHAMPM_2011', 'I_PAUVRE50M_2011', 'I_PAUVRE60M_2011', 'DEP_2014', 'DEPCOM_2014', 'I_PAUVRE50M_2014', 'I_PAUVRE60M_2014', 'TYPE_FISC_2011', 'TYPE_FISC_2014']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [143]:
fiscr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 790495 entries, 0 to 808323
Columns: 202 entries, ID_FISC_LOG_DIFF to TYPE_FISC_2014
dtypes: float64(74), int64(116), object(12)
memory usage: 1.2+ GB


In [144]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)

now:  2019-01-31 15:00:01.872914
execution_time:  0:02:57.638302
